In [8]:
import json
import stanza
import os

In [2]:
def load_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
        return data

In [5]:
nlp = stanza.Pipeline('uk')

2020-06-04 12:56:09 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-06-04 12:56:09 INFO: Use device: gpu
2020-06-04 12:56:09 INFO: Loading: tokenize
2020-06-04 12:56:11 INFO: Loading: mwt
2020-06-04 12:56:11 INFO: Loading: pos
2020-06-04 12:56:12 INFO: Loading: lemma
2020-06-04 12:56:12 INFO: Loading: depparse
2020-06-04 12:56:14 INFO: Done loading processors!


In [22]:
doc = nlp("Привітний персонал")
doc.sentences[0].to_dict()

[{'id': '1',
  'text': 'Привітний',
  'lemma': 'привітний',
  'upos': 'ADJ',
  'xpos': 'Ao-msnf',
  'feats': 'Case=Nom|Gender=Masc|Number=Sing',
  'head': 2,
  'deprel': 'amod',
  'misc': 'start_char=0|end_char=9'},
 {'id': '2',
  'text': 'персонал',
  'lemma': 'персонал',
  'upos': 'NOUN',
  'xpos': 'Ncmsny',
  'feats': 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing',
  'head': 0,
  'deprel': 'root',
  'misc': 'start_char=10|end_char=18'}]

In [40]:
def get_nouns(nlp, text: str):
    nouns = []
    doc = nlp(text)
    for sent in doc.sentences:
        for token in sent.to_dict():
            if 'upos' in token and token['upos'] == 'NOUN':
                nouns.append([token['text'], token['lemma']])
    return nouns

In [13]:
path_dir1 = "google/lviv"
path_dir2 = "google/kyiv"

In [14]:
files = []
for path_dir in [path_dir1, path_dir2]:
    for i in os.listdir(path_dir):
        if i.endswith('.json'):
            files.append(f"{path_dir}/{i}")

In [15]:
len(files)

526

In [16]:
hotel_review_dict = dict()
for file in files:
    hotel_review_dict[file] = load_json(file)

In [64]:
topic_text_dict = dict()
count_all_reviews = 0
count_reviews = 0
for hotel, review in hotel_review_dict.items():
#     print(review.keys())
    for item in review['reviews']:
        count_all_reviews += 1
        if item['should_be_more_details'] is False:
            count_reviews += 1
            for topic, text in item['topics'].items():
                if text is not None:
                    if topic not in topic_text_dict:
                        topic_text_dict[topic] = [text]
                    else:
                        topic_text_dict[topic].append(text)


In [65]:
for k, v in topic_text_dict.items():
    print(k, len(v))

Bathroom and toiletries 987
Cleanliness 4226
Sleep 1300
Room amenities 1009
Parking 519
Property 6900
Public transit 583
Nightlife 251
Kitchen 903
Location 3393
Service 4758
Room entertainment 76
Atmosphere 3426
Wi-Fi 144
Accessibility 112
Breakfast 1419
Restaurant 946
Food and Beverage 1864
Air conditioning 142
Wellness 229
Spa 180
Gym 45
Fitness 180
Business 1037
Family friendly 115
Couple friendly 180
Nature and outdoor activities 190
Safety 55
Pool 63
Hot tub 11
Pets 14
Bar or lounge 21
Beach 1


In [66]:
with open('google/topic_texts.json', 'w') as f:
    json.dump(topic_text_dict, f)

In [19]:
topic_text_dict['Location']

[' Чудовий хостел недалеко від центру міста.',
 ' Доречі в центрі міста . А ще можна піти в ТЦ , який приблизно 500 м від хостелу .',
 'Плюсами є ціна і розташування.',
 'Файні номери по доступній ціні, поруч з центром міста.',
 'Недалеко від центру. ',
 ' До Оперного 10 хвилин ходу.',
 'Чудовий хостел, недалеко від центру міста!',
 ' В 5 хвилинах  ходьби від центру.',
 'Дуже затишно і близько до центру😎',
 'Хостел із середніми цінами неподалік від центру старого міста Львів)',
 'Помірні ціни, поруч центру міста і гарни персонал!',
 'Блиск до центру міста',
 ' Дуже близько до центру.',
 ' Додаток близькість до центру',
 'Гарне місце, 15 хвилин до центру. ',
 'Плюс близько центр, все під рукою. ',
 'Відмінне місце розташування. ',
 'Недорого і до центру 10 хв ходьби',
 'Бомжі поруч і район не дуже',
 'Дешеві номери в центрі міста.',
 'Біля центру є ресторан.',
 'Розташування незручне, а так гаряча вода є, постіль чиста, жіночка приємна, все порадить',
 'Шумно, погане розташування',
 'Не

In [41]:
get_nouns(nlp, topic_text_dict['Location'][0])

[['хостел', 'хостел'], ['центру', 'центр'], ['міста', 'місто']]

In [42]:
def get_topic_nouns(nlp, topic_texts):
    topic_nouns = []
    for text in topic_texts:
        topic_nouns += get_nouns(nlp, text)
    return  topic_nouns

In [61]:
def get_topic_lemma_count_dict(topic_lemmas):
    topic_lemma_count = dict()
    for lemma in topic_lemmas:
        if lemma not in topic_lemma_count:
            topic_lemma_count[lemma] = 1
        else:
            topic_lemma_count[lemma] += 1
    topic_lemma_count = {k: v for k, v in sorted(topic_lemma_count.items(), key=lambda item: item[1], reverse=True)}
    return topic_lemma_count

In [43]:
location_nouns = get_topic_nouns(nlp, topic_text_dict['Location'])

In [44]:
location_nouns

[['хостел', 'хостел'],
 ['центру', 'центр'],
 ['міста', 'місто'],
 ['центрі', 'центр'],
 ['міста', 'місто'],
 ['ТЦ', 'ТЦ'],
 ['м', 'м'],
 ['хостелу', 'хостел'],
 ['Плюсами', 'плюс'],
 ['ціна', 'ціна'],
 ['розташування', 'розташування'],
 ['номери', 'номер'],
 ['ціні', 'ціна'],
 ['центром', 'центр'],
 ['міста', 'місто'],
 ['центру', 'центр'],
 ['хвилин', 'хвилина'],
 ['ходу', 'хід'],
 ['хостел', 'хостел'],
 ['центру', 'центр'],
 ['міста', 'місто'],
 ['хвилинах', 'хвилина'],
 ['ходьби', 'ходьба'],
 ['центру', 'центр'],
 ['центру', 'центр'],
 ['Хостел', 'хостел'],
 ['цінами', 'ціна'],
 ['центру', 'центр'],
 ['міста', 'місто'],
 ['ціни', 'ціна'],
 ['центру', 'центр'],
 ['міста', 'місто'],
 ['персонал', 'персонал'],
 ['Блиск', 'блиск'],
 ['центру', 'центр'],
 ['міста', 'місто'],
 ['центру', 'центр'],
 ['Додаток', 'додаток'],
 ['близькість', 'близькість'],
 ['центру', 'центр'],
 ['місце', 'місце'],
 ['хвилин', 'хвилина'],
 ['центру', 'центр'],
 ['центр', 'центр'],
 ['рукою', 'рука'],
 ['місц

In [49]:
location_lemmas_set = set([item[1] for item in location_nouns])

In [50]:
location_lemmas = [item[1] for item in location_nouns]

In [55]:
location_lemma_count = get_topic_lemma_count_dict(location_lemmas)

In [53]:
location_lemma_count

{'розташування': 1560,
 'центр': 794,
 'місце': 659,
 'готель': 563,
 'місто': 395,
 'персонал': 344,
 'номер': 236,
 'вокзал': 236,
 'метро': 227,
 'ціна': 220,
 'хостел': 124,
 'локація': 118,
 'сніданок': 105,
 'сервіс': 97,
 'хвилина': 90,
 'обслуговування': 78,
 'кімната': 57,
 'кухня': 54,
 'станція': 50,
 'магазин': 44,
 'якість': 43,
 'супер': 42,
 'місцерозташування': 42,
 'хв.': 40,
 'аеропорт': 39,
 'ресторан': 35,
 'площа': 35,
 'ходьба': 34,
 'транспорт': 34,
 'умова': 34,
 'ринок': 32,
 'атмосфера': 31,
 'місцезнаходження': 31,
 'плюс': 30,
 'інтер’єр': 30,
 'парк': 29,
 'район': 27,
 'чистота': 27,
 'парковка': 25,
 'проживання': 24,
 'рівень': 24,
 'номера': 24,
 'зупинка': 24,
 'розміщення': 23,
 'вид': 23,
 'вікно': 21,
 'мінус': 20,
 'доступність': 20,
 'розв’язка': 19,
 'жд.': 19,
 'дорога': 18,
 'гуртожиток': 18,
 'пам’ятка': 18,
 'ліжко': 17,
 'комфорт': 17,
 'частина': 17,
 'зручність': 17,
 'співвідношення': 16,
 'вулиця': 16,
 'серце': 15,
 'раз': 15,
 'ремонт'

In [60]:
service_nouns = get_topic_nouns(nlp, topic_text_dict['Service'])

In [62]:
service_lemma_count = get_topic_lemma_count_dict( [item[1] for item in service_nouns])

In [63]:
service_lemma_count

{'персонал': 2979,
 'обслуговування': 703,
 'сервіс': 619,
 'готель': 595,
 'номер': 455,
 'кухня': 297,
 'місце': 288,
 'розташування': 288,
 'ціна': 277,
 'рівень': 173,
 'хостел': 145,
 'центр': 124,
 'атмосфера': 109,
 'кімната': 106,
 'ресторан': 98,
 'інтер’єр': 85,
 'господар': 83,
 'послуга': 72,
 'рецепція': 69,
 'якість': 68,
 'умова': 68,
 'супер': 67,
 'чистота': 66,
 'номера': 64,
 'місто': 59,
 'їжа': 58,
 'працівник': 57,
 'людина': 46,
 'адміністратор': 44,
 'вид': 42,
 'власник': 41,
 'сніданок': 41,
 'співробітник': 40,
 'ліжко': 39,
 'заклад': 37,
 'висота': 36,
 'гостинність': 35,
 'клієнт': 34,
 'ремонт': 31,
 'комфорт': 28,
 'дівчина': 28,
 'гість': 28,
 'проживання': 26,
 'ставлення': 26,
 'прийом': 26,
 'враження': 25,
 'вікно': 24,
 'локація': 23,
 'раз': 22,
 'зірка': 22,
 'зал': 22,
 'відпочинок': 21,
 'робота': 21,
 'зала': 21,
 'офіціант': 21,
 'отель': 20,
 'страва': 20,
 'час': 20,
 'вокзал': 20,
 'територія': 20,
 'мова': 20,
 'метро': 20,
 'гроші': 19,
